In [1]:
import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "consumer_disputed"

# Feature name, feature dimensionality
ONE_HOT_FEATURES = {
    "product": 11,
    "sub_product": 45,
    "company_response": 5,
    "state": 60,
    "issue": 90
}

# Feature name, bucket count
BUCKET_FEATURES = {
    "zip_code": 10
}

# Feature name, value is unused
TEXT_FEATURES = {
    "consumer_complaint_narrative": None
}

In [2]:
def transformed_name(key):
    return key + '_xf'

In [3]:
# helper function to convert sparse to dense features

def fill_in_missing(x):
    default_value = '' if x.dtype == tf.string or to_string else 0
    if type(x) == tf.SparseTensor:
        x = tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0],1]),default_value)
        return tf.squeeze(x,axis=1)

In [4]:
# helper function converts a given index to a one-hot encoded representation

def convert_num_to_one_hot(label_tensor,num_labels=2):
    one_hot_tensor = tf.one_hot(label_tensor,num_labels)
    return tf.reshape(one_hot_tensor, [-1, num_labels])

In [5]:
# helper function to convert zip codes represented as strings to float values

def convert_zip_code(zip_code):
    if zip_code == '':
        zip_code = "00000"
    zip_code = tf.strings.regex_replace(zip_code, r'X{0,5}',"0")
    zip_code = tf.strings.to_number(zip_code, out_type=tf.float32)
    return zip_code

In [6]:
def preprocessing_fn(inputs):
    outputs = {}
    for key in ONE_HOT_FEATURES.keys():
        dim = ONE_HOT_FEATURES[key]
        index = tft.compute_and_apply_vocabulary(
            fill_in_missing(inputs[key]), top_k=dim + 1)
        outputs[transformed_name(key)] = convert_num_to_one_hot(
            index, num_labels=dim + 1)
        ...
        return outputs

In [7]:
for key, bucket_count in BUCKET_FEATURES.items():
    temp_feature = tft.bucketize(
            convert_zip_code(fill_in_missing(inputs[key])),
            bucket_count,
            always_return_num_quantiles=False)
    outputs[transformed_name(key)] = convert_num_to_one_hot(
            temp_feature,
            num_labels=bucket_count + 1)

NameError: name 'inputs' is not defined

In [8]:
for key in TEXT_FEATURES.keys():
    outputs[transformed_name(key)] = \
        fill_in_missing(inputs[key])
    
outputs[transformed_name(LABEL_KEY)] = fill_in_missing(inputs[LABEL_KEY])

NameError: name 'inputs' is not defined

## THIS IS NOT SUPPOSED TO BE A JUPYTER NOTEBOOK - THIS IS module.py and should be passed into transform as:

In [ ]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("module.py"))
context.run(transform)